In [1]:
import numpy as np 
import pandas as pd

from gsdmm import MovieGroupProcess

from gensim.models.coherencemodel import CoherenceModel
from gensim import corpora, models
from gensim.utils import simple_preprocess
import gensim
from gensim.models.ldamulticore import LdaMulticore
import re
import spacy

from gensim.models import Phrases
from gensim.models.phrases import Phraser

C:\Users\Alkan\anaconda3\lib\site-packages\torch\cuda\__init__.py:52: UserWarning: CUDA initialization: The NVIDIA driver on your system is too old (found version 9010). Please update your GPU driver by downloading and installing a new version from the URL: http://www.nvidia.com/Download/index.aspx Alternatively, go to: https://pytorch.org to install a PyTorch version that has been compiled with your version of the CUDA driver. (Triggered internally at  ..\c10\cuda\CUDAFunctions.cpp:109.)
  return torch._C._cuda_getDeviceCount() > 0


In [2]:
df=pd.read_csv("predictedsentiment_syrians_logistic.csv")

In [3]:
df

,Unnamed: 0,Tweet,PredictedSentiment
0,0,suriye türkiye sosyal demogratifyapısınıbozmuş...,Negative
1,1,devlet suriye güven bölge aşama ilerlemek yapm...,Neutral
2,2,sadece suriye değil afgan pakistan gitmek tali...,Negative
3,3,adana mülteci kamp yemek beğenmek suriye olay ...,Neutral
4,4,gün biri hülya han yahut bir yakın suriye afga...,Neutral
...,...,...,...
35880,35880,faşizm akp yarmak li çete soymak akp saray yol...,Negative
35881,35881,e devlet te isim köyişi değişik gelmek suriye ...,Negative
35882,35882,suriye dönmek demek,Neutral
35883,35883,suriye,Neutral


In [4]:
negative = df[df["PredictedSentiment"] == "Positive"]
negative

,Unnamed: 0,Tweet,PredictedSentiment
23,23,alman kadar olamadık savaş kaçmak gitmek o çağ...,Positive
114,114,hep gelmek gideceksiniz ilk can sıkıntı gemi y...,Positive
128,128,bir türk ırk değil söz dikkat ediniztürk ırk o...,Positive
179,179,suriye suriye suriye dönüyor site ediyorlarmış...,Positive
629,629,suriye afgan sabetay yahudi vermek zarar verme...,Positive
...,...,...,...
35353,35353,suriye onur sap saman karışmak filistinli siyo...,Positive
35610,35610,suriye sadece müslüman olmak böyle söylüyorsun...,Positive
35652,35652,ümit özdağ aşırı türkçümilliyetçi iş arapça ta...,Positive
35757,35757,suriye arap ırk dünya üzeri kanser bir hücre g...,Positive


In [5]:
def make_n_grams(texts):
    bigram = gensim.models.Phrases(texts, min_count=5, threshold=100)  # higher threshold fewer phrases.
    bigram_mod = gensim.models.phrases.Phraser(bigram)
    trigram = gensim.models.Phrases(bigram[texts], threshold=100)
    trigram_mod = gensim.models.phrases.Phraser(trigram)
    bigrams_text = [bigram_mod[doc] for doc in texts]
    trigrams_text =  [trigram_mod[bigram_mod[doc]] for doc in bigrams_text]
    return trigrams_text

In [6]:

negative['Tweet'] = negative.Tweet.values.tolist()

def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence)))

C:\Users\Alkan\AppData\Local\Temp\ipykernel_22340\1752566637.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  negative['Tweet'] = negative.Tweet.values.tolist()


In [7]:
tokens_tweets = list(sent_to_words(negative['Tweet']))

In [8]:
np.random.seed(0) #for reproducability

In [9]:
mgp = MovieGroupProcess(K=15, alpha=0.01, beta=0.01, n_iters=30) 

vocab = set(x for tweet in tokens_tweets for x in tweet)
n_terms = len(vocab)
model = mgp.fit(tokens_tweets, n_terms)

In stage 0: transferred 318 clusters with 15 clusters populated
In stage 1: transferred 145 clusters with 15 clusters populated
In stage 2: transferred 75 clusters with 15 clusters populated
In stage 3: transferred 61 clusters with 15 clusters populated
In stage 4: transferred 39 clusters with 15 clusters populated
In stage 5: transferred 30 clusters with 15 clusters populated
In stage 6: transferred 32 clusters with 15 clusters populated
In stage 7: transferred 26 clusters with 15 clusters populated
In stage 8: transferred 25 clusters with 15 clusters populated
In stage 9: transferred 25 clusters with 15 clusters populated
In stage 10: transferred 17 clusters with 15 clusters populated
In stage 11: transferred 30 clusters with 15 clusters populated
In stage 12: transferred 24 clusters with 15 clusters populated
In stage 13: transferred 23 clusters with 15 clusters populated
In stage 14: transferred 18 clusters with 15 clusters populated
In stage 15: transferred 25 clusters with 15 clu

In [10]:
def top_words(cluster_word_distribution, top_cluster, values):
    for cluster in top_cluster:
        sort_dicts =sorted(mgp.cluster_word_distribution[cluster].items(), key=lambda k: k[1], reverse=True)[:values]
        print("\nCluster %s : %s"%(cluster,sort_dicts))

In [11]:
doc_count = np.array(mgp.cluster_doc_count)
print('Number of documents per topic :', doc_count)

# topics sorted by the number of document they are allocated to
top_index = doc_count.argsort()[-15:][::-1]
print('\nMost important clusters (by number of docs inside):', top_index)
# show the top 5 words in term frequency for each cluster 
top_words(mgp.cluster_word_distribution, top_index, 20)

Number of documents per topic : [51 10 13 13 84 37 12 31  8 34 26 41 47 11 12]

Most important clusters (by number of docs inside): [ 4  0 12 11  5  9  7 10  3  2 14  6 13  1  8]

Cluster 4 : [('suriye', 135), ('demek', 66), ('bir', 62), ('ırk', 52), ('olmak', 37), ('savaşmak', 37), ('gitmek', 29), ('insan', 28), ('yapmak', 27), ('değil', 26), ('yok', 23), ('mi', 22), ('var', 22), ('türkiye', 21), ('kalmak', 21), ('afgan', 19), ('kadar', 18), ('kendi', 17), ('bun', 16), ('pkk', 16)]

Cluster 0 : [('suriye', 64), ('savaşmak', 64), ('bir', 35), ('ırk', 27), ('olmak', 24), ('ukrayna', 24), ('mi', 22), ('değil', 14), ('demek', 14), ('müslüman', 14), ('biz', 14), ('savaş', 13), ('yapmak', 12), ('var', 12), ('kendi', 12), ('ön', 12), ('insan', 11), ('yok', 11), ('iç', 11), ('gelmek', 10)]

Cluster 12 : [('suriye', 66), ('demek', 41), ('bir', 35), ('ırk', 32), ('olmak', 21), ('müslüman', 19), ('savaşmak', 19), ('afgan', 17), ('gitmek', 17), ('kardeş', 15), ('kadar', 14), ('insan', 13), ('ülke

In [100]:
gsdmm_clusters=["Cluster 11 : [('afgan', 3160), ('suriye', 2092), ('ülke', 1977), ('bir', 1581), ('olmak', 978), ('mülteci', 944), ('desen', 621), ('demek', 607), ('değil', 563), ('gelmek', 535), ('var', 523), ('türkiye', 491), ('bun', 474), ('türk', 425), ('mi', 417), ('biz', 416), ('on', 412), ('yok', 389), ('kendi', 332), ('kadar', 330)”],

["Cluster 1 : [('afgan', 1903), ('ülke', 1242), ('suriye', 1005), ('bir', 778), ('mülteci', 537), ('olmak', 529), ('çocuk', 376), ('mi', 359), ('demek', 352), ('değil', 313), ('biz', 303), ('bun', 292), ('gelmek', 288), ('ben', 286), ('on', 282), ('desen', 277), ('kendi', 273), ('kız', 260), ('yok', 252), ('vatan', 240)”],

["Cluster 13 : [('afgan', 200), ('suriye', 152), ('iş', 87), ('bir', 74), ('olmak', 72), ('ülke', 64), ('yok', 59), ('çalışmak', 41), ('desen', 37), ('ev', 37), ('biz', 36), ('var', 36), ('değil', 33), ('do', 31), ('on', 30), ('demek', 29), ('türk', 29), ('mülteci', 28), ('gelmek', 28), ('mi', 27)”],

["Cluster 5 : [('türkiye', 212), ('bütün', 107), ('iade', 107), ('karışmak', 105), ('mülteci', 100), ('geri', 94), ('mahalle', 94), ('derhâl', 94), ('göçmen', 90), ('alınıp', 88), ('sığınmak', 79), ('kötü', 77), ('karışacak', 76), ('afgan', 73), ('hudutnamustur', 69), ('suriye', 67), ('arap', 66), ('afrikalı', 59), ('edilsinleryoksa', 58), ('ran', 47)”],

["luster 3 : [('yıl', 185), ('geri', 161), ('çocuk', 129), ('afgan', 101), ('bizde', 99), ('suriye', 83), ('özcan', 79), ('bun', 73), ('tanju', 73), ('karı', 67), ('göz', 67), ('yaşın', 66), ('olabilecek', 66), ('sevmek', 65), ('yavru', 64), ('entegre', 60), ('biz', 56), ('top', 56), ('edeceksiniz', 50), ('bakıyorlar', 49)],

["Cluster 14 : [('mülteci', 97), ('afgan', 92), ('ayşegül', 81), ('bir', 69), ('yaşın', 56), ('ülke', 40), ('tarafından', 40), ('çocuk', 37), ('tecavüz', 37), ('taş', 36), ('kız', 35), ('hayat', 33), ('aydın', 28), ('olmak', 27), ('uğrayan', 27), ('saldırı', 26), ('istemek', 25), ('baş', 25), ('suriye', 24), ('kafa', 22)”],

["Cluster 2 : [('ülke', 88), ('afgan', 83), ('suriye', 58), ('mülteci', 52), ('sığınmak', 31), ('istemek', 27), ('irak', 25), ('vs', 23), ('göçmek', 22), ('türkiye', 21), ('kendi', 20), ('sınırdışı', 18), ('arap', 17), ('suri', 17), ('yok', 16), ('ler', 15), ('olmak', 14), ('türk', 14), ('bir', 13), ('vatandaş', 12)”],

["Cluster 8 : [('afgan', 77), ('suriye', 76), ('sığınmak', 34), ('ülke', 34), ('iş', 30), ('ordu', 28), ('haç', 27), ('millet', 26), ('para', 26), ('el', 26), ('aşı', 26), ('alıyor', 26), ('ekmek', 26), ('proje', 21), ('gitmek', 19), ('istemek', 19), ('dönmek', 19), ('lar', 12), ('göz', 10), ('halk', 10)”],

["Cluster 0 : [('afgan', 76), ('mülteci', 53), ('bir', 44), ('kız', 32), ('ayşegül', 29), ('olmak', 26), ('ülke', 23), ('tarafından', 22), ('baş', 21), ('desen', 19), ('genç', 19), ('taş', 19), ('suriye', 17), ('yaşın', 17), ('tecavüz', 15), ('iç', 13), ('hayat', 13), ('türkiye', 12), ('mi', 12), ('sınır', 11)”],

["Cluster 12 : [('afgan', 74), ('türkiye', 44), ('olmak', 36), ('sağlık', 34), ('asker', 29), ('çocuk', 28), ('traş', 28), ('kadın', 28), ('suriye', 25), ('âdeta', 25), ('gelir', 22), ('ülke', 19), ('masal', 19), ('genç', 15), ('taliban', 14), ('mülteci', 14), ('avukat', 14), ('korkunç', 14), ('yaka', 13), ('desen', 12)”],

["Cluster 10 : [('afgan', 69), ('türkiye', 63), ('olmak', 44), ('ülke', 42), ('sağlık', 39), ('âdeta', 39), ('asker', 39), ('traş', 39), ('afganistan', 35), ('çocuk', 34), ('kadın', 34), ('görünüm', 31), ('girmek', 30), ('sığınmak', 30), ('militan', 30), ('genç', 22), ('idare', 22), ('ediyörpütin', 22), ('hudutnamustur', 21), ('gelirtürkiyeyi', 17)”],

["Cluster 6 : [('afgan', 50), ('ülke', 24), ('gi', 22), ('değil', 20), ('on', 17), ('emperyalizm', 14), ('ni', 14), ('mi', 12), ('millet', 12), ('bi', 12), ('bir', 11), ('mazlum', 11), ('ede', 11), ('li', 10), ('suri', 10), ('istemek', 10), ('yeli', 10), ('bun', 9), ('ler', 9), ('çocuk', 8)”],

["Cluster 9 : [('afgan', 36), ('bilemek', 29), ('herkes', 28), ('olmak', 26), ('fakir', 21), ('suriye', 18), ('yine', 18), ('savaşmak', 18), ('mülteci', 14), ('türkiye', 14), ('asker', 14), ('genç', 13), ('bir', 10), ('kadın', 10), ('sağlık', 10), ('almak', 10), ('çocuk', 10), ('bitmek', 10), ('gem', 10), ('kaptan', 10)”],

["Cluster 7 : [('bir', 42), ('afgan', 24), ('olmak', 17), ('suriye', 12), ('önlem', 12), ('ülke', 11), ('sonra', 10), ('yan', 10), ('biz', 9), ('yok', 9), ('mülteci', 9), ('milyon', 7), ('can', 7), ('türkiye', 6), ('yanmak', 6), ('ad', 6), ('bilemek', 6), ('aile', 6), ('patlamak', 6), ('demek', 5)”],

["Cluster 4 : [('afgan', 17), ('suriye', 11), ('ülke', 9), ('bir', 9), ('mülteci', 8), ('türkiye', 7), ('pakistan', 7), ('olmak', 6), ('milyon', 6), ('fır', 6), ('işgal', 5), ('değil', 4), ('demek', 4), ('durum', 4), ('vatandaş', 4), ('kul', 4), ('uyuşturmak', 4), ('dava', 4), ('cevap', 4), ('türk', 3)"]


SyntaxError: EOL while scanning string literal (624936307.py, line 1)

In [ ]:
gsdmm_clusters = [["Cluster 2 : [('afganlar', 987), ('ve', 657), ('suriyeliler', 519), ('bu', 510), ('bir', 502), ('de', 311), ('mülteci', 289), ('da', 261), ('afgan', 259), ('suriyeli', 256), ('ne', 213), ('için', 198), ('var', 198), ('yok', 191), ('türkiye', 179), ('gibi', 160), ('çok', 154), ('değil', 146), ('daha', 134), ('ülke', 127)"],

["Cluster 10 : [('afganlar', 563), ('bir', 339), ('bu', 312), ('ve', 298), ('mülteci', 277), ('suriyeliler', 274), ('afgan', 255), ('suriyeli', 227), ('ne', 225), ('da', 162), ('de', 161), ('yok', 123), ('var', 103), ('değil', 101), ('daha', 100), ('gibi', 99), ('kadar', 96), ('için', 92), ('çok', 90), ('ama', 89)"],

["Cluster 8 : [('afganlar', 623), ('bu', 395), ('ve', 320), ('bir', 273), ('afgan', 239), ('mülteci', 211), ('suriyeliler', 200), ('ne', 181), ('da', 155), ('de', 153), ('için', 140), ('değil', 126), ('ama', 97), ('suriyeli', 96), ('yok', 93), ('var', 91), ('ya', 87), ('kadar', 85), ('kendi', 85), ('neden', 83)"],

["Cluster 14 : [('afganlar', 402), ('ve', 345), ('bu', 242), ('bir', 242), ('mülteci', 227), ('afgan', 216), ('suriyeli', 186), ('suriyeliler', 168), ('da', 130), ('de', 128), ('ne', 119), ('değil', 108), ('türkiye', 103), ('için', 97), ('gibi', 81), ('var', 79), ('kadar', 75), ('çok', 72), ('https', 60), ('co', 60)"],

["Cluster 0 : [('afganlar', 442), ('ve', 260), ('bir', 241), ('suriyeliler', 204), ('bu', 188), ('da', 143), ('de', 142), ('afgan', 121), ('ne', 116), ('için', 104), ('mülteci', 101), ('suriyeli', 98), ('türkiye', 90), ('ama', 84), ('değil', 73), ('var', 65), ('gibi', 62), ('yok', 62), ('daha', 61), ('çok', 57)"],

["Cluster 7 : [('afganlar', 375), ('bu', 184), ('suriyeliler', 172), ('bir', 151), ('ve', 150), ('ne', 104), ('de', 85), ('mülteci', 84), ('da', 82), ('afgan', 80), ('suriyeli', 80), ('var', 76), ('yok', 75), ('ülke', 67), ('kadar', 60), ('kendi', 59), ('daha', 58), ('ama', 57), ('sonra', 55), ('ya', 50)"],

["Cluster 6 : [('afganlar', 290), ('suriyeliler', 162), ('bu', 107), ('ve', 106), ('bir', 92), ('ne', 86), ('mülteci', 61), ('yok', 60), ('var', 57), ('suriyeli', 54), ('dolar', 54), ('afgan', 49), ('mi', 49), ('kendi', 45), ('de', 44), ('da', 44), ('çok', 37), ('biz', 37), ('bi', 36), ('daha', 32)"],

["Cluster 11 : [('afganlar', 227), ('bu', 107), ('ve', 94), ('suriyeliler', 80), ('bir', 60), ('mi', 55), ('bi', 52), ('ne', 48), ('de', 44), ('icin', 41), ('da', 40), ('herkes', 37), ('ler', 33), ('ama', 32), ('suri', 32), ('ye', 30), ('ki', 29), ('biliyor', 29), ('daha', 28), ('yeli', 28)"],

["Cluster 4 : [('afgan', 176), ('mülteci', 171), ('bir', 148), ('ayşegül', 116), ('afganlar', 74), ('bu', 68), ('tarafından', 64), ('ve', 63), ('https', 62), ('co', 62), ('taşla', 53), ('de', 50), ('yaşındaki', 50), ('tecavüz', 42), ('kız', 41), ('için', 37), ('da', 36), ('suriyeli', 33), ('istemiyorum', 33), ('türk', 32)"],

["Cluster 9 : [('afganlar', 150), ('ve', 99), ('suriyeliler', 66), ('bu', 63), ('bir', 62), ('de', 52), ('da', 37), ('suriyeli', 36), ('çok', 34), ('gibi', 32), ('ne', 32), ('afgan', 30), ('var', 29), ('ama', 27), ('türk', 27), ('mülteci', 25), ('daha', 24), ('suriye', 24), ('https', 23), ('co', 23)"],

["Cluster 13 : [('afganlar', 138), ('ve', 138), ('suriyeliler', 93), ('sığınmacılar', 38), ('bu', 37), ('mülteci', 30), ('afgan', 30), ('ülkesine', 30), ('milletin', 28), ('de', 28), ('bir', 27), ('ekmeğini', 27), ('haçlı', 27), ('ordusu', 27), ('da', 26), ('parasını', 26), ('aşını', 26), ('işini', 26), ('elinden', 26), ('alıyor', 26)"],

["Cluster 12 : [('yıl', 187), ('afganlar', 123), ('bizden', 99), ('suriyeliler', 94), ('geride', 93), ('özcan', 92), ('tanju', 79), ('gerideydi', 78), ('bizim', 69), ('diye', 68), ('sevdiğimiz', 66), ('karısı', 66), ('olabilecek', 66), ('yavrum', 65), ('çocuğum', 64), ('nasıl', 62), ('https', 62), ('co', 62), ('entegre', 61), ('edeceksiniz', 60)"],

["Cluster 3 : [('türkiye', 192), ('ve', 118), ('afganlar', 107), ('geri', 96), ('derhal', 96), ('suriyeliler', 94), ('bütün', 94), ('araplar', 94), ('alınıp', 94), ('iade', 94), ('yi', 93), ('karıştırmadan', 91), ('afrikalılar', 91), ('iranlılar', 91), ('mahallemizden', 91), ('edilsinler', 91), ('mülteciler', 80), ('göçmenler', 77), ('yoksa', 77), ('çok', 77)"],

["Cluster 1 : [('afganlar', 76), ('ve', 58), ('bir', 54), ('türkiye', 42), ('afgan', 41), ('mülteci', 34), ('ye', 31), ('bu', 22), ('genç', 21), ('https', 21), ('co', 21), ('de', 20), ('ne', 20), ('asker', 20), ('neden', 19), ('suriyeliler', 18), ('suriyeli', 18), ('kadınları', 18), ('olmayan', 18), ('ile', 17)"],

["Cluster 5 : [('türkiye', 84), ('afganlar', 77), ('ve', 61), ('ye', 61), ('putin', 44), ('neden', 43), ('adeta', 43), ('genç', 43), ('sağlıklı', 43), ('asker', 43), ('traşlı', 43), ('çocukları', 43), ('kadınları', 43), ('olmayan', 43), ('gelir', 40), ('sınırnamustur', 36), ('daki', 35), ('hudutnamustur', 34), ('ülkemize', 32), ('sığınmacı', 32)"]]

In [ ]:
gsdmm_clusters = pd.DataFrame.from_dict(gsdmm_clusters)

In [ ]:
gsdmm_clusters

In [ ]:
gsdmm_clusters.rename(columns={0: "Dominant Keywords for GSDMM Clusters"}, inplace=True)

In [ ]:
gsdmm_clusters[['Cluster No.(Importance)','Dominant Keywords:GSDMM']] = gsdmm_clusters["Dominant Keywords for GSDMM Clusters"].str.split(":",expand=True,)
gsdmm_clusters

In [ ]:
# I didn`t rename the clusters

topic_dict = {}
topic_names = ['type 1',
               'type 2',
               'type 3',
               'type 4',
               'type 5',
               'type 6',
               'type 7',
               'type 8',
               'type 9',
               'type 10',
               'type 11',
               'type 12',
               'type 13',
               'type 14',
               'type 15'
              ]
for i, topic_num in enumerate(top_index):
    topic_dict[topic_num]=topic_names[i] 

In [ ]:
def create_topics_dataframe(negative_text=negative.Tweet,  mgp=mgp, threshold=0.3, topic_dict=topic_dict, lemma_text=tokens_tweets):
    result = pd.DataFrame(columns=['Text', 'Topic', 'Rating', 'Lemma-text'])
    for i, text in enumerate(negative_text):
        result.at[i, 'Text'] = text
        #result.at[i, 'Rating'] = data.Rating[i]
        result.at[i, 'Lemma-text'] = lemma_text[i]
        prob = mgp.choose_best_label(tokens_tweets[i])
        if prob[1] >= threshold:
            result.at[i, 'Topic'] = topic_dict[prob[0]]
        else:
            result.at[i, 'Topic'] = 'Other'
    return result

In [ ]:
result = create_topics_dataframe(negative_text=negative.Tweet, mgp=mgp, threshold=0.3, topic_dict=topic_dict, lemma_text=tokens_tweets)
result.head(16)

In [ ]:
#VISUALIZATION

In [ ]:
import plotly.express as px

fig = px.pie(result, names='Topic',  title="Figure 4. GSDMM for Abortion Tweets with Negative Sentiment", color_discrete_sequence=px.colors.sequential.Burg)
fig.show()

In [ ]:
result['len'] = result.Text.apply(lambda row: len(row.split()))
result.head()

In [ ]:
result

In [ ]:
#WORDCLOUDS

In [ ]:
result['Lemma_text'] = result['Lemma-text'].apply(lambda row: ' '.join(row))

In [ ]:
result = result.drop('Lemma-text', axis=1)

In [ ]:
!pip install wordcloud

In [ ]:
from wordcloud import WordCloud 

In [ ]:
my_stopwords = ["afgan", "böyle", "bir", "bu", "şey", "mi", "mı", "var", "ki", "ama", "diye", "her", "tamam","kadar", "işte", "çi", "çki", "bi", "rbia", "ne", "mıdır", "midir", "da", "le", "el","için", "de","belli", "ve", "değil", "gibi", "ile"]

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
def create_WordCloud(data, title=None):
    wordcloud = WordCloud(width = 500, height = 500,
                          background_color ='white',
                          min_font_size = 15, stopwords = my_stopwords
                         ).generate(" ".join(data.values))
                      
    plt.figure(figsize = (5, 5), facecolor = None) 
    plt.imshow(wordcloud, interpolation='bilinear') 
    plt.axis("off") 
    plt.tight_layout(pad = 0) 
    plt.title(title,fontsize=20)
    plt.show() 

In [ ]:
create_WordCloud(result['Lemma_text'].loc[result.Topic == 'type 1'], title="Most used words in negative sentiment tweets")

In [ ]:
create_WordCloud(result['Lemma_text'].loc[result.Topic == 'type 2'], title="Most used words in negative sentiment tweets")